In [1]:
import numpy as np
import numpy.typing as npt
from typing import Union, List, Callable, Any, Tuple, Literal

# Interpolation and Polynomial Approximation

## Lagrange Interpolation
If $x_0,x_1,\cdots,x_n$ are $n+1$ distinct numbers and $f$ is a function whose values are given at these numbers, then a unique polynomial $P(x)$ of degree at most $n$ exists with
$$f(x_k)=P(x_k),\,\,\,\text{for each }k=0,1,\cdots,n$$
This polynomial is given by
$$P(x)=f(x_0)L_{n, 0}(x)+\cdots+f(x_n)L_{n, n}(x)=\sum^n_{k=0}f(x_k)L_{n, k}(x)$$
where for each $k=0,1,\cdots,n$
$$L_{n, k}(x)=\frac{(x-x_0)(x-x_1)\cdots(x-x_{k-1})(x-x_{k+1})\cdots(x-x_n)}{(x_k-x_0)(x_k-x_1)\cdots(x_k-x_{k-1})(x_k-x_{k+1})\cdots(x_k-x_n)}=\prod^n_{\substack{i=0\\i\neq k}}\frac{(x-x_i)}{(x_k-x_i)}$$

In [2]:
def LagrangeInterpolation(func: Union[Callable[[Union[float, int]], float],
                                      np.ndarray[Union[float, int], Any],
                                      List[Union[int, float]]],
                          x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
                          x: Union[float, int]
                          ) -> float:
    appro = 0
    for i, x_i in enumerate(x_):
        l_i = 1
        for k, x_k in enumerate(x_):
            if k != i:
                l_i *= (x - x_k) / (x_i - x_k)
        appro += func(x_i) * l_i if callable(func) else func[i] * l_i
    return appro

## Neville's Method
$$ Q_{i, j}=\frac{(x-x_{i-j})Q_{i, j-1}-(x-x_i)Q_{i-1, j-1}}{x_i-x_{i-j}}$$
where $P(x)=Q_{n, n}$

In [3]:
def Neville(func: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
            x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
            x: Union[float, int]
            ) -> float:
    n = len(x_)
    Q = np.zeros((n, n))
    for idx in range(n):
        Q[idx][0] = func(x_[idx]) if callable(func) else func[idx]
    for idx in range(1, n):
        for jdx in range(1, idx + 1):
            numerator = (x - x_[idx - jdx]) * Q[idx][jdx - 1] - (x - x_[idx]) * Q[idx - 1][jdx - 1]
            denominator = x_[idx] - x_[idx - jdx]
            Q[idx][jdx] = numerator / denominator
    return Q[-1, -1]

## Newton's Dividend-Difference Formula
$$P_n(x)=F_{0, 0}+\sum^n_{i=1}F_{i, i}\prod_{j=0}^{i-1}(x-x_j)$$
where $,\forall i=0,1\cdots,n$
$$F_{i, 0}=f(x_i)$$
$$F_{i, j}=\frac{F_{i, j-1}-F_{i-1, j-1}}{x_i-x_{i-j}}$$

In [4]:
def NewtonDiff(func: Union[Callable[[Union[float, int]], float],
                           np.ndarray[Union[float, int], Any],
                           List[Union[int, float]]],
            x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
            return_Q: bool=False,
            ) -> Union[Callable[[Union[float, int]], float],
                       Tuple[Callable[[Union[float, int]], float], np.ndarray[Union[float, int], Any]]]:
    
    f_call = callable(func)
    n = len(x_)
    Q = np.zeros((n, n))
    
    for idx in range(n):
        Q[idx][0] = func(x_[idx]) if f_call else func[idx]

    for idx in range(1, n):
        for jdx in range(1, idx + 1):
            Q[idx][jdx] = (Q[idx][jdx - 1] -  Q[idx - 1][jdx - 1]) / (x_[idx] - x_[idx - jdx])

    def interpolation(x: float) -> float:
        appro = Q[0][0]
        for idx in range(1, n):
            prod = 1
            for jdx in range(idx):
                prod *= (x - x_[jdx])
            appro += Q[idx][idx] * prod 
        return appro

    return interpolation if not return_Q else (interpolation, Q)


In [5]:
X = [1.0, 1.3, 1.6, 1.9, 2.2]
F = [0.7651977, 0.6200860, 0.4554022, 0.2818186, 0.1103623]
x = 1.5
poly = NewtonDiff(F, X)
poly(x)

0.5118199942386833

## Hermite Interpolation

In [6]:
def Hermite(func: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
            dfunc: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
            x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
            return_Q: bool=False
            ) -> Union[Callable[[Union[float, int]], float],
                       Tuple[Callable[[Union[float, int]], float], np.ndarray[Union[float, int], Any]]]:
    f_call = callable(func)
    df_call = callable(dfunc)
    n = len(x_)
    
    z = np.zeros(2 * n)
    Q = np.zeros((2 * n, 2 * n))

    for idx in range(n):
        z[2 * idx] = x_[idx]
        z[2 * idx + 1] = x_[idx]
        Q[2 * idx, 0] = func(x_[idx]) if f_call else func[idx]
        Q[2 * idx + 1, 0] = func(x_[idx]) if f_call else func[idx]
        Q[2 * idx + 1, 1] = dfunc(x_[idx]) if df_call else dfunc[idx]
        if idx:
            Q[2 * idx, 1] = (Q[2 * idx, 0] - Q[2 * idx - 1, 0]) / (z[2 * idx] - z[2 * idx - 1])
    for idx in range(2, 2 * n):
        for jdx in range(2, idx + 1):
            Q[idx][jdx] = (Q[idx][jdx - 1] -  Q[idx - 1][jdx - 1]) / (z[idx] - z[idx - jdx])
            
    def interpolation(x:float) -> float:
        appro = Q[0][0]
        for idx in range(1, 2 * n):
            prod = 1
            for jdx in range(idx):
                prod *= (x - z[jdx])
            appro += Q[idx][idx] * prod
        return appro

    return interpolation if not return_Q else (interpolation, Q)

In [7]:
X = [0.1, 0.2, 0.3, 0.4]
F = [-0.62049958, -0.28398668, 0.00660095, 0.24842440]
dF = [3.58502082, 3.14033271, 2.66668043, 2.16529366]
x = 0.34
poly = Hermite(F, dF, X)
poly(x)

0.10933658972789766

In [8]:
X = [1.3, 1.6, 1.9]
F = [0.6200860, 0.4554022, 0.2818186]
dF = [-0.5220232, -0.5698959, -0.5811571]
x = 1.5
poly = Hermite(F, dF, X)
poly(x)

0.5118277017283951

## Natural Cubic Spline
Construct the cubic spline interpolant $S$ for the function $f$, defined at the numbers $x_0<x_1<\cdots<x_n$, satisfying $S''(x_0)=S''(x_n)=0$
where
$$ S(x)=S_j(x)=a_j+b_j(x-x_j)+c_j(x-x_j)^2+d_j(x-x_j)^3,x_j\leq x\leq x_{j+1}$$

In [9]:
def NaturalCubic(func: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
                 x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
            ) -> np.ndarray[Union[float, int], Any]:
    '''
    Returns:
    - Numpy array with shape (len(x_) - 1, 4) where each row represents the coefficients (a, b, c, d)
      of a cubic spline segment, from the first to the last segment.
    '''
    n = len(x_)
    coef = np.zeros((4, n))
    f_call = callable(func)

    for idx in range(n):
        coef[0][idx] = func(x_[idx]) if f_call else func[idx]

    h = np.diff(x_)
    alpha = np.zeros((n))
    for idx in range(1, n - 1):
        diff = func(x_[idx + 1]) - func(x_[idx]) if f_call else func[idx + 1] - func[idx]
        diff_ = func(x_[idx]) - func(x_[idx - 1]) if f_call else func[idx] - func[idx - 1]
        alpha[idx] = (3 * diff / h[idx]) - (3 * diff_ / h[idx - 1])

    l = np.ones((n, ))
    mu = np.zeros((n, ))
    z = np.zeros((n, ))

    for idx in range(1, n - 1):
        l[idx] = 2 * (x_[idx + 1] - x_[idx - 1]) - h[idx - 1] * mu[idx - 1]
        mu[idx] = h[idx] / l[idx]
        z[idx] = (alpha[idx] - h[idx - 1] * z[idx - 1]) / l[idx]
    for jdx in range(n - 2, -1, -1):
        coef[2][jdx] = z[jdx] - mu[jdx] * coef[2][jdx + 1]
        coef[1][jdx] = (coef[0][jdx + 1] - coef[0][jdx]) / h[jdx] - h[jdx] * (coef[2][jdx + 1] + 2 * coef[2][jdx]) / 3
        coef[3][jdx] = (coef[2][jdx + 1] - coef[2][jdx]) / (3 * h[jdx])
    return coef[:, :n - 1].T

In [10]:
import math
F = [1, math.exp(1), math.exp(2), math.exp(3)]
X = [0, 1, 2, 3]
NaturalCubic(F, X)

array([[ 1.        ,  1.46599761,  0.        ,  0.25228421],
       [ 2.71828183,  2.22285026,  0.75685264,  1.69107137],
       [ 7.3890561 ,  8.80976965,  5.83006675, -1.94335558]])

## Clamped Cubic Spline
To construct the cubic spline interpolant $S$ for the function $f$ defined at the numbers $x_0<x_1<\cdots<x_n$, satisfying $S'(x_0)=f'(x_0)$ and $S'(x_n)=f'(x_n)$
where
$$ S(x)=S_j(x)=a_j+b_j(x-x_j)+c_j(x-x_j)^2+d_j(x-x_j)^3,x_j\leq x\leq x_{j+1}$$

In [11]:
def ClampedCubic(func: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
                 dfunc: Union[Callable[[Union[float, int]], float],
                        np.ndarray[Union[float, int], Any],
                        List[Union[int, float]]],
                 x_: Union[np.ndarray[Union[float, int], Any], List[Union[int, float]]],
            ) -> np.ndarray[Union[float, int], Any]:
    '''
    Returns:
    - Numpy array with shape (len(x_) - 1, 4) where each row represents the coefficients (a, b, c, d)
      of a cubic spline segment, from the first to the last segment.
    '''
    n = len(x_)
    coef = np.zeros((4, n))
    f_call = callable(func)
    df_call = callable(func)

    df = dfunc(x_[0]) if df_call else dfunc[0]
    df_ = dfunc(x_[n - 1]) if df_call else dfunc[1]

    for idx in range(n):
        coef[0][idx] = func(x_[idx]) if f_call else func[idx]

    h = np.diff(x_)
    alpha = np.zeros((n))
    alpha[0] = 3 * (coef[0][1] - coef[0][0]) / h[0] - 3 * df
    alpha[n - 1] = 3 * (df_ - (coef[0][n - 1] - coef[0][n - 2]) / h[n - 2])

    for idx in range(1, n - 1):
        diff = func(x_[idx + 1]) - func(x_[idx]) if f_call else func[idx + 1] - func[idx]
        diff_ = func(x_[idx]) - func(x_[idx - 1]) if f_call else func[idx] - func[idx - 1]
        alpha[idx] = (3 * diff / h[idx]) - (3 * diff_ / h[idx - 1])

    l = np.zeros((n, ))
    mu = np.zeros((n, ))
    z = np.zeros((n, ))
    l[0] = 2 * h[0]
    mu[0] = 0.5
    z[0] = alpha[0] / l[0]

    for idx in range(1, n - 1):
        l[idx] = 2 * (x_[idx + 1] - x_[idx - 1]) - h[idx - 1] * mu[idx - 1]
        mu[idx] = h[idx] / l[idx]
        z[idx] = (alpha[idx] - h[idx - 1] * z[idx - 1]) / l[idx]

    l[n - 1] = h[n - 2] * (2 - mu[n - 2])
    z[n - 1] = (alpha[n - 1] - h[n - 2] * z[n - 2]) / l[n - 1]
    coef[2][n - 1] = z[n - 1]

    for jdx in range(n - 2, -1, -1):
        coef[2][jdx] = z[jdx] - mu[jdx] * coef[2][jdx + 1]
        coef[1][jdx] = (coef[0][jdx + 1] - coef[0][jdx]) / h[jdx] - h[jdx] * (coef[2][jdx + 1] + 2 * coef[2][jdx]) / 3
        coef[3][jdx] = (coef[2][jdx + 1] - coef[2][jdx]) / (3 * h[jdx])

    return coef[:, :n - 1].T

In [12]:
import math
F = [1, math.exp(1), math.exp(2), math.exp(3)]
dF = [1, math.exp(3)]
X = [0, 1, 2, 3]
ClampedCubic(F, dF, X)

array([[1.        , 1.        , 0.4446825 , 0.27359933],
       [2.71828183, 2.71016299, 1.26548049, 0.69513079],
       [7.3890561 , 7.32651634, 3.35087286, 2.01909162]])

#